In [19]:
# !pip install plotly
# !pip install nbformat

In [18]:
import gc
# import kaggle_evaluation.cmi_inference_server
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.io as pio
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F


from glob import glob
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm import tqdm

!mkdir output
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pio.renderers.default = 'iframe'

�T�u�f�B���N�g���܂��̓t�@�C�� output �͊��ɑ��݂��܂��B


In [20]:
class config:
    BATCH_SIZE_TEST = 1
    NUM_WORKERS = 0 # multiprocessing.cpu_count()
    PRINT_FREQ = 20
    SEED = 20

class paths:
    OUTPUT_DIR = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\output"
    TEST_CSV = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\test.csv"
    TEST_DEMOGRAPHICS = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\test_demographics.csv"
    TRAIN_CSV = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\train.csv"
    TRAIN_DEMOGRAPHICS = "C:\\Users\\konno\\SynologyDrive\\datasciense\\projects_foler\\1_kaggle\\CMI\\cmi-detect-behavior-with-sensor-data\\train_demographics.csv"

In [21]:
def format_for_scoring(df_preds: pd.DataFrame) ->tuple[pd.DataFrame, pd.DataFrame]: 
    solution = df_preds[["sequence_id", "y_true"]].copy()
    solution.columns = ["id", "gesture"]
    solution["gesture"] = solution["gesture"].map(num_to_label)

    submission = df_preds[["sequence_id", "y_pred"]].copy()
    submission.columns = ["id", "gesture"]
    submission["gesture"] = submission["gesture"].map(num_to_label)
    
    return solution, submission


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    

def sep():
    print("—"*100)


label_to_num = {
    'Above ear - pull hair': 0,  # < ------- TARGETS START
    'Cheek - pinch skin': 1,
    'Eyebrow - pull hair': 2,
    'Eyelash - pull hair': 3,
    'Forehead - pull hairline': 4,
    'Forehead - scratch': 5,
    'Neck - pinch skin': 6,
    'Neck - scratch': 7,  # < ------- TARGETS END
    'Drink from bottle/cup': 8,  # < ------- NON-TARGETS START
    'Feel around in tray and pull out an object': 8,
    'Glasses on/off': 8,
    'Pinch knee/leg skin': 8,
    'Pull air toward your face': 8,
    'Scratch knee/leg skin': 8,
    'Text on phone': 8,
    'Wave hello': 8,
    'Write name in air': 8,
    'Write name on leg': 8  # < ------- NON-TARGETS END
}
type_to_num = {"Target": 1, "Non-Target":0}
num_to_label = {v: k for k, v in label_to_num.items()}
num_to_type = {v: k for k, v in type_to_num.items()}
seed_everything(config.SEED)

In [22]:
df_test = pd.read_csv(paths.TEST_CSV)
df_test_demographics = pd.read_csv(paths.TEST_DEMOGRAPHICS)

print(f"Test dataframe shape: {df_test.shape}"), sep()
print(f"Test demographics dataframe shape: {df_test_demographics.shape}")

display(df_test.head())
display(df_test_demographics.head())

Test dataframe shape: (107, 336)
————————————————————————————————————————————————————————————————————————————————————————————————————
Test demographics dataframe shape: (2, 8)


,row_id,sequence_id,sequence_counter,subject,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000001_000000,SEQ_000001,0,SUBJ_055840,9.039062,5.261719,0.800781,0.367188,-0.397400,-0.629028,...,97.0,87.0,206.0,-1.0,195.0,-1.0,-1.0,-1.0,111.0,-1.0
1,SEQ_000001_000001,SEQ_000001,1,SUBJ_055840,9.421875,3.460938,-1.113281,0.353882,-0.507141,-0.652710,...,175.0,158.0,-1.0,-1.0,-1.0,-1.0,-1.0,211.0,187.0,178.0
2,SEQ_000001_000002,SEQ_000001,2,SUBJ_055840,10.160156,2.082031,-3.871094,0.384094,-0.532104,-0.639648,...,-1.0,160.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,197.0,177.0
3,SEQ_000001_000003,SEQ_000001,3,SUBJ_055840,9.773438,1.355469,-4.371094,0.387756,-0.531982,-0.634033,...,-1.0,160.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,197.0,183.0
4,SEQ_000001_000004,SEQ_000001,4,SUBJ_055840,9.195312,1.011719,-3.222656,0.382751,-0.534180,-0.638367,...,-1.0,163.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,200.0,173.0


,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_016452,1,25,1,1,165.0,52,23.0
1,SUBJ_055840,0,13,0,1,177.0,52,27.0


In [23]:
def min_max_scale(arr: np.ndarray) -> np.ndarray:
    min_vals = np.nanmin(arr, axis=0)
    max_vals = np.nanmax(arr, axis=0)
    ranges = np.where(max_vals - min_vals == 0, 1, max_vals - min_vals)
    scaled = (arr - min_vals) / ranges
    return scaled


def standard_scale(arr: np.ndarray) -> np.ndarray:
    means = np.nanmean(arr, axis=0)
    stds = np.nanstd(arr, axis=0)
    stds = np.where(stds == 0, 1, stds)  # Prevent division by zero for constant columns
    scaled = (arr - means) / stds
    return scaled


def pad_or_truncate(
    arr: np.ndarray,
    max_length: int = 200,
    pad_value: int = 0,
    mode: str = "random"  # "regular" or "random"
) -> np.ndarray:
    L, D = arr.shape

    if L > max_length:
        return arr[:max_length, :]

    elif L < max_length:
        if mode == "regular":
            padding = np.full((max_length - L, D), pad_value)
            return np.vstack((arr, padding))
        
        elif mode == "random":
            total_padding = max_length - L
            pad_start = np.random.randint(0, total_padding + 1)
            pad_end = total_padding - pad_start

            start_padding = np.full((pad_start, D), pad_value)
            end_padding = np.full((pad_end, D), pad_value)

            return np.vstack((start_padding, arr, end_padding))
        
        else:
            raise ValueError(f"Unknown mode: {mode}. Use 'regular' or 'random'.")

    else:
        return arr

imu_cols = ["acc_x", "acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
X_test  = []

for sequence_id in tqdm(df_test.sequence_id.unique()):
    ds = df_test[df_test["sequence_id"] == sequence_id]
    X = ds[imu_cols].values
    X = pad_or_truncate(X)
    X = np.concatenate((standard_scale(X[:, 0:3]), X[:, 3:]), axis=1)
    X = np.where(np.isnan(X), 0.0, X)  # fill NaNs
    X_test.append(X)

X_test = np.array(X_test)

100%|██████████| 2/2 [00:00<00:00, 335.69it/s]


In [24]:
class CustomDataset(Dataset):
    def __init__(
        self, config, df: pd.DataFrame, X: np.ndarray
    ): 
        
        self.config = config
        self.df = df
        self.X = X
        self.indexes = self.df.sequence_id.unique()
        
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.indexes)
        
    def __getitem__(self, index):
        """
        Get one item.
        """
        sequence_id = self.indexes[index]
        X = self.X[index]
        output = {
            "X": torch.tensor(X, dtype=torch.float32),
            "sequence_id": sequence_id
        }
        return output 

In [26]:
test_dataset = CustomDataset(config, df_test, X_test)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE_TEST,
    shuffle=False,
    num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
)
idx = np.random.choice(range(0, len(test_dataset)))
cols = imu_cols
X = test_dataset[idx]["X"]
sequence_id = test_dataset[idx]["sequence_id"]
N = X.shape[0]
df = pd.DataFrame(X, columns=cols)
df['step'] = range(N)
df_melted = df.melt(id_vars='step', var_name='sequence', value_name='value')

fig = px.line(
    df_melted,
    x='step',
    y='value',
    color='sequence',
    title=f"Sequences for {sequence_id}",
    template='plotly_dark',
    width=800,
    height=600
)

fig.show(renderer='browser')

In [27]:
class Wave_Block(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, dilation_rates: int, kernel_size: int = 3):
        """
        WaveNet building block.
        :param in_channels: number of input channels.
        :param out_channels: number of output channels.
        :param dilation_rates: how many levels of dilations are used.
        :param kernel_size: size of the convolving kernel.
        """
        super(Wave_Block, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        # First conv: (B, in_channels, L) -> (B, out_channels, L)
        self.convs.append(
            nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=True)
        )
        
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            # Filter conv: (B, out_channels, L) -> (B, out_channels, L)
            self.filter_convs.append(
                nn.Conv1d(
                    out_channels, out_channels, kernel_size=kernel_size,
                    padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate)
            )
            # Gate conv: (B, out_channels, L) -> (B, out_channels, L)
            self.gate_convs.append(
                nn.Conv1d(
                    out_channels, out_channels, kernel_size=kernel_size,
                    padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate)
            )
            # Residual conv: (B, out_channels, L) -> (B, out_channels, L)
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1, bias=True))
        
        for i in range(len(self.convs)):
            nn.init.xavier_uniform_(self.convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.convs[i].bias)

        for i in range(len(self.filter_convs)):
            nn.init.xavier_uniform_(self.filter_convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.filter_convs[i].bias)

        for i in range(len(self.gate_convs)):
            nn.init.xavier_uniform_(self.gate_convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.gate_convs[i].bias)

    def forward(self, x: torch.Tensor):
        # x: (B, in_channels, L)
        x = self.convs[0](x)  # (B, in_channels, L) -> (B, out_channels, L)
        res = x  # res: (B, out_channels, L)
        for i in range(self.num_rates):
            tanh_out = torch.tanh(self.filter_convs[i](x))  # (B, out_channels, L) -> (B, out_channels, L)
            sigmoid_out = torch.sigmoid(self.gate_convs[i](x)) # (B, out_channels, L) -> (B, out_channels, L)
            x = tanh_out * sigmoid_out  # (B, out_channels, L) * (B, out_channels, L) -> (B, out_channels, L)
            x = self.convs[i + 1](x) # (B, out_channels, L) -> (B, out_channels, L)
            res = res + x  # (B, out_channels, L) + (B, out_channels, L) -> (B, out_channels, L)
        return res  # (B, out_channels, L)
    
class WaveNet(nn.Module):
    def __init__(self, input_channels: int = 1, kernel_size: int = 3):
        super(WaveNet, self).__init__()
        self.model = nn.Sequential(
                Wave_Block(input_channels, 32, 12, kernel_size),   # (B, input_channels, L) -> (B, 8, L)
                Wave_Block(32, 64, 8, kernel_size),                # (B, 8, L) -> (B, 16, L)
                Wave_Block(64, 128, 4, kernel_size),               # (B, 16, L) -> (B, 32, L)
                Wave_Block(128, 256, 1, kernel_size),                # (B, 32, L) -> (B, 64, L)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, L, input_channels) - typical input format
        x = x.permute(0, 2, 1)  # (B, L, input_channels) -> (B, input_channels, L)
        output = self.model(x)  # (B, input_channels, L) -> (B, 64, L)
        return output  # (B, 64, L)


class TemporalAttentionPooling(nn.Module):
    def __init__(self, input_channels, hidden_dim=64):
        super(TemporalAttentionPooling, self).__init__()
        self.attn = nn.Sequential(
            nn.Conv1d(input_channels, hidden_dim, kernel_size=1),
            nn.Tanh(),
            nn.Conv1d(hidden_dim, 1, kernel_size=1)
        )

    def forward(self, x):
        """
        x: (B, C, L)
        returns: (B, C)
        """
        # Compute attention scores
        attn_scores = self.attn(x)  # (B, 1, L)
        attn_weights = F.softmax(attn_scores, dim=-1)  # (B, 1, L)

        # Weighted sum over time
        weighted = x * attn_weights  # (B, C, L)
        pooled = weighted.sum(dim=-1)  # (B, C)

        return pooled



class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        self.wavenet1 = WaveNet(input_channels=3)  # WaveNet: (B, input_channels, L) -> (B, 64, L)
        self.wavenet2 = WaveNet(input_channels=4)  # WaveNet: (B, input_channels, L) -> (B, 64, L)
        self.config = config
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)  # (B, 64, L) -> (B, 64, 1)
        self.dropout = 0.2
        self.head_1 = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(512, 256), # (B, 64) -> (B, 64)
            nn.BatchNorm1d(256),  # (B, 64) -> (B, 64)
            nn.ReLU(),  # (B, 64) -> (B, 64)
            nn.Dropout(self.dropout),  # (B, 64) -> (B, 64)
            nn.Linear(256, num_classes)  # (B, 64) -> (B, num_classes)
        )
        self.head_2 = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(512, 256), # (B, 64) -> (B, 64)
            nn.BatchNorm1d(256),  # (B, 64) -> (B, 64)
            nn.ReLU(),  # (B, 64) -> (B, 64)
            nn.Dropout(self.dropout),  # (B, 64) -> (B, 64)
            nn.Linear(256, 1)  # (B, 64) -> (B, num_classes)
        )
        
    def forward(self, x: torch.Tensor):
        """
        Forward pass.
        """
        # x: (B, L, input_channels) - typical input format
        x1 = self.wavenet1(x[:, :, 0:3])  # (B, L, input_channels) -> (B, 64, L)
        x1 = self.global_avg_pooling(x1)  # (B, 64, L) -> (B, 64, 1)
        x2 = self.wavenet2(x[:, :, 3:])  # (B, L, input_channels) -> (B, 64, L)
        x2 = self.global_avg_pooling(x2)  # (B, 64, L) -> (B, 64, 1)
        y = torch.concatenate([x1, x2], axis=1) # (B, 128)
        z1 = self.head_1(y)  # (B, 64) -> (B, num_classes)
        z2 = self.head_2(y)  # (B, 64) -> (B, num_classes)
        return z1, z2  # (B, num_classes)

model = CustomModel(num_classes=9)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 2829226


In [32]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_test = sequence.to_pandas()
    imu_cols = ["acc_x", "acc_y", "acc_z", "rot_w", "rot_x", "rot_y", "rot_z"]
    X_test, y_test, y_hard_test = [], [], []
    
    for sequence_id in tqdm(df_test.sequence_id.unique()):
        ds = df_test[df_test["sequence_id"] == sequence_id]
        X = ds[imu_cols].values
        X = pad_or_truncate(X)
        X = np.concatenate((standard_scale(X[:, 0:3]), X[:, 3:]), axis=1)
        X = np.where(np.isnan(X), 0.0, X)  # fill NaNs
        X_test.append(X)

    X_test = np.array(X_test)
    model_paths = glob("/kaggle/input/cmi-wavenet/*.pth")
    all_preds = []
    for model_path in model_paths:
        test_dataset = valid_dataset = CustomDataset(config, df_test, X_test)
        test_loader = DataLoader(
            test_dataset,
            batch_size=config.BATCH_SIZE_TEST,
            shuffle=False,
            num_workers=config.NUM_WORKERS, 
            pin_memory=True, drop_last=False
        )
        model = CustomModel(num_classes=9)
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint)
        model.to(device)
        model.eval()
        softmax = nn.Softmax(dim=1)
        
        with tqdm(test_loader, unit="test_batch", desc='Test') as tqdm_test_loader:
            for step, batch in enumerate(tqdm_test_loader):
                X = batch.pop("X").to(device)
                batch_size = X.size(0)
                with torch.no_grad():
                    y_preds, y_preds_hard = model(X)
                y_preds = softmax(y_preds).to('cpu').numpy()
                all_preds.append(y_preds)
    all_preds = np.concatenate(all_preds)
    all_preds = np.argmax(all_preds.mean(axis=0)).item()
    prediction = num_to_label[all_preds]
    return prediction

# Load data (update the paths to your local copies)
test_csv_path = paths.TEST_CSV
test_demo_path = paths.TEST_DEMOGRAPHICS

# Load the test data
test_df = pd.read_csv(test_csv_path)
demographics_df = pd.read_csv(test_demo_path)

test_pl = pl.from_pandas(test_df)

# Run predictions
results = []

# Loop over each unique sequence_id
for sequence_id in test_df['sequence_id'].unique():
    # Extract data for this sequence
    seq_df = test_pl.filter(pl.col("sequence_id") == sequence_id)

    # Call your model's prediction function
    pred_label = predict(seq_df, demographics_df)

    # Save the result
    results.append((sequence_id, pred_label))

# Convert results to DataFrame and save
submission_df = pd.DataFrame(results, columns=["sequence_id", "prediction"])
submission_df.to_csv("submission.csv", index=False)

print("✅ Inference complete. Submission saved to submission.csv")

# Launch inference server
# inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway(
#         data_paths=(
#             '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
#             '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
#         )
#     )

100%|██████████| 1/1 [00:00<00:00, 1026.25it/s]


ValueError: need at least one array to concatenate